In [5]:
# load packages and data

library(data.table) 
library(ggplot2)
library(dplyr) # if not installed locally, in command prompt use "conda install -c r r-dplyr"

loadDir = '../Data/' # set directory where data will be loaded from
loadFName = 'Make 18_full.csv' # set file name of Qualtrics csv
rawData <- data.table::fread(input = paste(loadDir, loadFName, sep=''))

# also load in data from AMT to match MIDs (to download: on AMT site > Review Results > Download CSV)
amtIDs = data.table::fread(input = paste(loadDir, 'Make 18_full_AMT.csv' , sep=''), select='WorkerId') 

dataStartRow = 75 # set the row number when full experiment starts (used to exclude MIDs from past pilot runs)


Warning message in data.table::fread(input = paste(loadDir, "Make 18_full_AMT.csv", :
"Detected 30 column names but the data has 28 columns. Filling rows automatically. Set fill=TRUE explicitly to avoid this warning."

In [6]:
head(rawData)

StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,q1_txt,q2_txt,q3_txt,q4_txt,q5_txt,surveycode,MID,p1_txt,p2_txt,Random
Start Date,End Date,Response Type,IP Address,Progress,Duration (in seconds),Finished,Recorded Date,Response ID,Recipient Last Name,...,q1_txt,q2_txt,q3_txt,q4_txt,q5_txt,surveycode,MID,p1_txt,p2_txt,Random
"{""""ImportId"""":""""startDate"""",""""timeZone"""":""""America/Denver""""}","{""""ImportId"""":""""endDate"""",""""timeZone"""":""""America/Denver""""}","{""""ImportId"""":""""status""""}","{""""ImportId"""":""""ipAddress""""}","{""""ImportId"""":""""progress""""}","{""""ImportId"""":""""duration""""}","{""""ImportId"""":""""finished""""}","{""""ImportId"""":""""recordedDate"""",""""timeZone"""":""""America/Denver""""}","{""""ImportId"""":""""_recordId""""}","{""""ImportId"""":""""recipientLastName""""}",...,"{""""ImportId"""":""""q1_txt""""}","{""""ImportId"""":""""q2_txt""""}","{""""ImportId"""":""""q3_txt""""}","{""""ImportId"""":""""q4_txt""""}","{""""ImportId"""":""""q5_txt""""}","{""""ImportId"""":""""surveycode""""}","{""""ImportId"""":""""MID""""}","{""""ImportId"""":""""p1_txt""""}","{""""ImportId"""":""""p2_txt""""}","{""""ImportId"""":""""Random""""}"
2021-07-09 22:36:19,2021-07-09 22:37:51,Survey Preview,,100,91,True,2021-07-09 22:37:54,R_2YhLYlwEGxfIgCh,,...,NaN(5+7)*(10-8),NaN(11+13)*(6-5),NaN3+7*2,NaN10+8-7-5,NaN2+3+5+8,9348,,,,2
2021-07-09 22:39:36,2021-07-09 22:53:16,IP Address,157.51.62.60,100,820,True,2021-07-09 22:53:17,R_2e4IILDq5qEipzH,,...,NaN10+8/(6-7),NaN13+5,NaN7*2+(1*4),,NaN8*3-(5+7),1441,A33Z442WJQ8ADC,,,2
2021-07-09 22:38:07,2021-07-09 22:54:39,IP Address,201.47.22.214,100,992,True,2021-07-09 22:54:40,R_2U5fS9EqxPKlcze,,...,NaN3*6*(8-7),NaN5*6*(11-13),NaN2*7*(4-1),NaN3*7*(7-2),~+3*8*(2-5),7876,A2825I2XO6KZEP,,,1
2021-07-09 22:54:07,2021-07-09 23:00:33,IP Address,192.154.206.9,100,385,True,2021-07-09 23:00:34,R_1k0TzyF95z6apGi,,...,NaN33.6,NaN11+13+5/6,NaN1/2*4*7,NaN7/3*7*2,NaN2/8*3+5,9853,A35T79LLCN429G,,,2


In [35]:
# basic removals

cleanedData = rawData[-(1:dataStartRow), ] # remove all data from previous pilot runs
cleanedData = cleanedData[MID %in% amtIDs[, WorkerId], ] # keep only entries with AMT worker IDs that match those on Amazon interface

# print repeated IPs and MIDs
print('Repeated IPs:')
data.table(table(c(cleanedData[, IPAddress], 000)))[N>1, ]
print('Repeated MIDs:')
data.table(table(c(cleanedData[, MID], 000)))[N>1, ]

# remove repeated IPs and MIDs
repeatedIPs = data.table(table(c(cleanedData[, IPAddress], 000)))[N>1, V1]
repeatedMIDs = data.table(table(c(cleanedData[, MID], 000)))[N>1, V1]

cleanedData = cleanedData[!(MID %in% repeatedMIDs), ] # remove all data from MechTurk IDs that appear more than once
cleanedData = cleanedData[!(IPAddress %in% repeatedIPs), ] # remove all data from IP addresses that appear more than once

'cleanedData Dimensions:'
dim(cleanedData)

[1] "Repeated IPs:"


V1,N
117.213.35.151,2
117.213.35.153,3
117.217.160.22,2
117.217.212.77,2
182.65.18.62,2
49.37.209.192,3
49.37.211.26,3
59.99.222.64,2


[1] "Repeated MIDs:"


V1,N
AFV9RX3T7ZHIB,2


[1] "cleanedData Dimensions:"

[1] 266 179

In [14]:
# check if any subjects did not finish the survey

cleanedData[Finished=='False', ]

Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"Warning message in cbind(parts$left, ellip_h, parts$right, deparse.level = 0L):
"number of rows of result is not a multiple of vector length (arg 2)"

StartDate,EndDate,Status,IPAddress,Progress,Duration (in seconds),Finished,RecordedDate,ResponseId,RecipientLastName,...,q1_txt,q2_txt,q3_txt,q4_txt,q5_txt,surveycode,MID,p1_txt,p2_txt,Random


In [15]:
# print all raw column names
names(cleanedData)

[1] "StartDate"                 "EndDate"                  
  [3] "Status"                    "IPAddress"                
  [5] "Progress"                  "Duration (in seconds)"    
  [7] "Finished"                  "RecordedDate"             
  [9] "ResponseId"                "RecipientLastName"        
 [11] "RecipientFirstName"        "RecipientEmail"           
 [13] "ExternalReference"         "LocationLatitude"         
 [15] "LocationLongitude"         "DistributionChannel"      
 [17] "UserLanguage"              "Q_RecaptchaScore"         
 [19] "Q_RecaptchaAssessmentName" "Q1.1_First Click"         
 [21] "Q1.1_Last Click"           "Q1.1_Page Submit"         
 [23] "Q1.1_Click Count"          "Q2.1"                     
 [25] "Q2.1_3_TEXT"               "Q2.2"                     
 [27] "Q2.3"                      "Q2.3_7_TEXT"              
 [29] "Q2.4"                      "Q2.5"                     
 [31] "Q2.6"                      "Q2.7_1"                   
 [33] "Q3.1_First Click"          "Q3.1_Last Click"          
 [35] "Q3.1_Page Submit"          "Q3.1_Click Count"         
 [37] "Q4.1_First Click"          "Q4.1_Last Click"          
 [39] "Q4.1_Page Submit"          "Q4.1_Click Count"         
 [41] "Q4.3_First Click"          "Q4.3_Last Click"          
 [43] "Q4.3_Page Submit"          "Q4.3_Click Count"         
 [45] "Q5.1_First Click"          "Q5.1_Last Click"          
 [47] "Q5.1_Page Submit"          "Q5.1_Click Count"         
 [49] "Q6.1_First Click"          "Q6.1_Last Click"          
 [51] "Q6.1_Page Submit"          "Q6.1_Click Count"         
 [53] "Q6.3_First Click"          "Q6.3_Last Click"          
 [55] "Q6.3_Page Submit"          "Q6.3_Click Count"         
 [57] "Q6.4_1"                    "Q6.5_1"                   
 [59] "Q6.6_First Click"          "Q6.6_Last Click"          
 [61] "Q6.6_Page Submit"          "Q6.6_Click Count"         
 [63] "Q6.8_First Click"          "Q6.8_Last Click"          
 [65] "Q6.8_Page Submit"          "Q6.8_Click Count"         
 [67] "Q6.9_1"                    "Q6.10_1"                  
 [69] "Q6.11_First Click"         "Q6.11_Last Click"         
 [71] "Q6.11_Page Submit"         "Q6.11_Click Count"        
 [73] "Q6.13_First Click"         "Q6.13_Last Click"         
 [75] "Q6.13_Page Submit"         "Q6.13_Click Count"        
 [77] "Q6.14_1"                   "Q6.15_1"                  
 [79] "Q6.16_First Click"         "Q6.16_Last Click"         
 [81] "Q6.16_Page Submit"         "Q6.16_Click Count"        
 [83] "Q6.18_First Click"         "Q6.18_Last Click"         
 [85] "Q6.18_Page Submit"         "Q6.18_Click Count"        
 [87] "Q6.19_1"                   "Q6.20_1"                  
 [89] "Q6.21_First Click"         "Q6.21_Last Click"         
 [91] "Q6.21_Page Submit"         "Q6.21_Click Count"        
 [93] "Q6.23_First Click"         "Q6.23_Last Click"         
 [95] "Q6.23_Page Submit"         "Q6.23_Click Count"        
 [97] "Q6.24_1"                   "Q6.25_1"                  
 [99] "Q7.1_First Click"          "Q7.1_Last Click"          
[101] "Q7.1_Page Submit"          "Q7.1_Click Count"         
[103] "Q8.1_First Click"          "Q8.1_Last Click"          
[105] "Q8.1_Page Submit"          "Q8.1_Click Count"         
[107] "Q8.3_First Click"          "Q8.3_Last Click"          
[109] "Q8.3_Page Submit"          "Q8.3_Click Count"         
[111] "Q8.4_1"                    "Q8.5_1"                   
[113] "Q8.6_First Click"          "Q8.6_Last Click"          
[115] "Q8.6_Page Submit"          "Q8.6_Click Count"         
[117] "Q8.8_First Click"          "Q8.8_Last Click"          
[119] "Q8.8_Page Submit"          "Q8.8_Click Count"         
[121] "Q8.9_1"                    "Q8.10_1"                  
[123] "Q8.11_First Click"         "Q8.11_Last Click"         
[125] "Q8.11_Page Submit"         "Q8.11_Click Count"        
[127] "Q8.13_First Click"         "Q8.13_Last Click"         
[129] "Q8.13_Page Submit"         

In [36]:
# coalesce control and treatment data into combined columns

numPuzzles = 5

defaultW <- getOption("warn") # suppress warnings that appear when coalescing
options(warn=-1)

# for each puzzle, coalesce the data (text and stress/confidence questions) from treatment and control into a single column
for (puzzleNum in c(1:numPuzzles)) {
    # create column names to get data from in each loop (i.e. for each puzzle)
    rt_colName_treat = paste('Q6.', toString(1+(puzzleNum-1)*5), '_Page Submit', sep='')
    rt_colName_control = paste('Q8.', toString(1+(puzzleNum-1)*5), '_Page Submit', sep='')
    rt_questions_colName_treat = paste('Q6.', toString(3+(puzzleNum-1)*5), '_Page Submit', sep='')
    rt_questions_colName_control = paste('Q8.', toString(3+(puzzleNum-1)*5), '_Page Submit', sep='')
    stress_colName_treat = paste('Q6.', toString(4+(puzzleNum-1)*5), '_1', sep='')
    stress_colName_control = paste('Q8.', toString(4+(puzzleNum-1)*5), '_1', sep='')
    confidence_colName_treat = paste('Q6.', toString(5+(puzzleNum-1)*5), '_1', sep='')
    confidence_colName_control = paste('Q8.', toString(5+(puzzleNum-1)*5), '_1', sep='')
    
    # coalesce the data for each puzzle question
    cleanedData[, paste('rt_q', puzzleNum , sep=''):=coalesce(as.numeric(unlist(cleanedData[, ..rt_colName_treat])), as.numeric(unlist(cleanedData[, ..rt_colName_control])))]
    cleanedData[, paste('rt_questions_q', puzzleNum , sep=''):=coalesce(as.numeric(unlist(cleanedData[, ..rt_questions_colName_treat])), as.numeric(unlist(cleanedData[, ..rt_questions_colName_control])))]
    cleanedData[, paste('stress_q', puzzleNum , sep=''):=coalesce(as.numeric(unlist(cleanedData[, ..stress_colName_treat])), as.numeric(unlist(cleanedData[, ..stress_colName_control])))]
    cleanedData[, paste('confidence_q', puzzleNum , sep=''):=coalesce(as.numeric(unlist(cleanedData[, ..confidence_colName_treat])), as.numeric(unlist(cleanedData[, ..confidence_colName_control])))]  
}

options(warn = defaultW)


# coalesce for treatment message page
cleanedData[, 'rt_treatMsg':=coalesce(as.numeric(unlist(cleanedData[, 'Q5.1_Page Submit'])), as.numeric(unlist(cleanedData[, 'Q7.1_Page Submit'])))]
cleanedData[, 'cc_treatMsg':=coalesce(as.numeric(unlist(cleanedData[, 'Q5.1_Click Count'])), as.numeric(unlist(cleanedData[, 'Q7.1_Click Count'])))]


## Rename Column Names (can be customized)

In [37]:
# rename columns

# rt = response time, cc = click count
##### change the key below if you want to use different column names:
renamedCols = c('IPAddress'='IP', 'Duration (in seconds)'='totTaskTime',
                'Q1.1_Page Submit'='rt_intro', 'Q1.1_Click Count'='cc_intro',
                'Q2.1'='gender', 'Q2.2'='age', 'Q2.3'='ethnicity', 'Q2.4'='education', 'Q2.5'='income', 'Q2.6'='intent', 'Q2.7_1'='mathEnjoyment',
                'Q3.1_Page Submit'='rt_instructions', 'Q3.1_Click Count'='cc_instructions', 
                'Q4.1_Page Submit'='rt_prac1', 'Q4.1_Click Count'='cc_prac1', 'p1_txt'='text_prac1',
                'Q4.3_Page Submit'='rt_prac2', 'Q4.3_Click Count'='cc_prac2', 'p2_txt'='text_prac2',
                'q1_txt'='text_q1', 'q2_txt'='text_q2', 'q3_txt'='text_q3', 'q4_txt'='text_q4', 'q5_txt'='text_q5', 
                'Q9.1'='postq_playedBefore', 'Q9.2_1'='postq_enjoy', 'Q9.3_1'='postq_difficulty', 'Q9.4_1'='postq_focus', 'Q9.5_1'='postq_pressure',
                'Q9.6'='postq_impact', 'Q11.1'='freeResponse',
                'Random'='treatGroup'
               )


# rename columns according to key above
for (colNum in 1:length(renamedCols)) {
    setnames(cleanedData, names(renamedCols[colNum]), toString(renamedCols[colNum]))
}

In [41]:
# subset to only columns that will be used for analysis

colsForAnalysis = c('totTaskTime', 'UserLanguage', 
                    'rt_intro', 'cc_intro',
                    'gender', 'age', 'ethnicity', 'education', 'income', 'intent', 'mathEnjoyment',
                    'rt_instructions', 'cc_instructions', 
                    'rt_prac1', 'cc_prac1', 'text_prac1',
                    'rt_prac2', 'cc_prac2', 'text_prac2',
                    'rt_treatMsg', 'cc_treatMsg',
                    'rt_q1', 'rt_questions_q1', 'stress_q1', 'confidence_q1',
                    'rt_q1', 'rt_questions_q2', 'stress_q2', 'confidence_q2',
                    'rt_q1', 'rt_questions_q3', 'stress_q3', 'confidence_q3',
                    'rt_q1', 'rt_questions_q4', 'stress_q4', 'confidence_q4',
                    'rt_q1', 'rt_questions_q5', 'stress_q5', 'confidence_q5',
                    'q1_score', 'q2_score', 'q3_score', 'q4_score', 'q5_score', 'final_score', 'true_earnings',
                    'postq_playedBefore', 'postq_enjoy', 'postq_difficulty', 'postq_focus', 'postq_pressure',
                    'postq_impact', 'freeResponse',
                    'treatGroup',
                    'MID'
                   )

cleanedData = cleanedData[ , ..colsForAnalysis] 

In [42]:
head(cleanedData)

totTaskTime,UserLanguage,rt_intro,cc_intro,gender,age,ethnicity,education,income,intent,...,true_earnings,postq_playedBefore,postq_enjoy,postq_difficulty,postq_focus,postq_pressure,postq_impact,freeResponse,treatGroup,MID
186,EN,5.049,0,Male,Over 65,White/ Caucasian,Bachelor's degree,"$25,000 - $50,000",Joy of solving math puzzles,...,0,Yes,5 Very enjoyable,5 Very Difficult,4,5 A lot of pressure,Knowing that my earnings depend on how many correct responses I submit,nice,1,A2MNDXS4E8LRJR
285,EN,5.537,0,Male,25-34,White/ Caucasian,Bachelor's degree,"$25,000 - $50,000",Joy of solving math puzzles,...,0,No,4,3 Neutral,4,4,Knowing that my earnings depend on how many correct responses I submit,No comments,1,A33S7UYXRIGYDR
252,EN,5.887,1,Female,45-54,White/ Caucasian,Bachelor's degree,"$50,000 - $100,000",Monetary incentive,...,0,Yes,3 Neutral,4,3 Neutral,4,Emotional high from answering puzzles correctly,nice,1,A3IKNT1N18NMNB
184,EN,2.275,0,Male,18-24,White/ Caucasian,Bachelor's degree,"$25,000 - $50,000",Joy of solving math puzzles,...,0.50,No,1 Did not enjoy,5 Very Difficult,2,2,Frustration from answering puzzles incorrectly,n/a,2,A3HZFB2JLF3JMY
273,EN,6.355,2,Male,35-44,White/ Caucasian,Bachelor's degree,"$50,000 - $100,000",Joy of solving math puzzles,...,0,No,4,4,3 Neutral,4,Knowing that my earnings depend on how many correct responses I submit,Very well,1,A18F0VDH08E0QY
321,EN,16.851,13,Male,25-34,White/ Caucasian,Bachelor's degree,"$50,000 - $100,000",Joy of solving math puzzles,...,0,Yes,4,5 Very Difficult,5 Very focused,4,Emotional high from answering puzzles correctly,GOOD,1,A3E2TLUKGTFGLO


### Function for checking if text input matches numbers given in puzzle
(will keep working on using this function to exclude subjects, but left it here if you guys want to use it)

In [19]:
# function for comparing inputted numbers with those given by the puzzle
# getting only the inputted numeric values is kind of contrived 
# because we need to differentiate between '10" as '10' and not '1' and '0'
# we do this by going through each character and checking if the character after each number is also a number or not

checkDigits = function (inputStr, puzzleNums) {
    allChars = unlist(strsplit(inputStr, split = ""))
    
    inputtedNums = c()

    defaultW <- getOption("warn") # suppress warnings that appear when coalescing
    options(warn=-1)
    
    for (charNum in 1:length(allChars)) {
        if (all(c(is.na(as.numeric(allChars[charNum - 1])), TRUE))) { # only collect numbers when previous character is not a number
            currChar = allChars[charNum]

            if (charNum == length(allChars)) { # no next character at end of list
                nextChar = NA
            } else {
                nextChar = allChars[charNum + 1]     
            }

            # numbers inputted next to each other are counted as a single number (eg. 10 is '10' and not '1' and '0')
            if (!is.na(as.numeric(currChar))) {
                if (!is.na(as.numeric(currChar)) & !is.na(as.numeric(nextChar))) {
                    inputtedNums = c(inputtedNums, as.numeric(paste(currChar, nextChar, sep='')))
                } else {
                    inputtedNums = c(inputtedNums, as.numeric(currChar))  
                }
            }
        }
    }

    options(warn = defaultW)
    
    # check if subject's input contains all 4 of the numbers given in the puzzle
    length(inputtedNums)==length(puzzleNums) & setequal(inputtedNums, puzzleNums)
}

In [21]:
allPuzzleNums = list('prac1' = c(2,10,9,10), 'prac2' = c(3,6,7,8),
                     'q1' = c(6,10,7,8), 'q2' = c(11,13,5,6), 'q3' = c(1,2,4,7), 'q4' = c(7,3,7,3), 'q5' = c(2,8,3,5)
                    )

subjInput = 'q~2*8+5-3'
puzzleNums = unlist(allPuzzleNums['q5'])

checkDigits(subjInput, puzzleNums)

[1] TRUE

In [44]:
apply(cleanedData[, 'text_prac1'], 1, checkDigits, puzzleNums=unlist(allPuzzleNums['prac1']))

[1] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [13] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE
 [25] FALSE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
 [37] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE
 [49] FALSE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE  TRUE FALSE FALSE  TRUE
 [61] FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE  TRUE FALSE FALSE FALSE
 [73] FALSE FALSE  TRUE FALSE  TRUE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE
 [85] FALSE FALSE  TRUE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE  TRUE
 [97] FALSE FALSE FALSE FALSE  TRUE  TRUE FALSE FALSE FALSE FALSE  TRUE  TRUE
[109]  TRUE FALSE FALSE FALSE  TRUE FALSE FALSE  TRUE FALSE  TRUE FALSE FALSE
[121] FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[133] FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE FALSE
[145] FALSE FALSE FALSE  TRUE  TRUE  TRUE FALSE FALSE  TRUE FALSE  TRUE FALSE
[157] FALSE  TRUE FALSE FALSE FALSE FALSE  TRUE  TRUE FALSE  TRUE  TRUE FALSE
[169] FALSE  TRUE  TRUE  TRUE FALSE FALSE  TRUE  TRUE FALSE FALSE FALSE FALSE
[181] FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE
[193]  TRUE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE  TRUE FALSE FALSE FALSE
[205] FALSE  TRUE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE
[217] FALSE FALSE FALSE  TRUE  TRUE  TRUE FALSE FALSE FALSE FALSE FALSE FALSE
[229] FALSE FALSE FALSE FALSE  TRUE FALSE  TRUE FALSE FALSE  TRUE FALSE FALSE
[241] FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE  TRUE FALSE  TRUE FALSE
[253] FALSE FALSE FALSE FALSE FALSE FALSE  TRUE FALSE FALSE FALSE FALSE  TRUE
[265] FALSE FALSE